<h1 align=center><font size = 5>Segmenting and Clustering Neighborhoods in Toronto:1</font></h1>

In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import requests

In [2]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
source = requests.get(url).text
soup=BeautifulSoup(source,'xml')

In [3]:
table1=soup.find('table')

In [4]:
#The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood.
column_name=['Postalcode','Borough','Neighborhood']
df = pd.DataFrame(columns = column_name)

In [12]:
table=soup.find('table')

<h5>Getting all values in tr and seperating each td within by ","</h5>

In [13]:
# Search all the postcode, borough, neighborhood 
for tr in table.find_all('tr'):
    row_data=[]
    for td in tr.find_all('td'):
        row_data.append(td.text.strip())
    if len(row_data)==3:
        df.loc[len(df)] = row_data

In [14]:
df.head()

,Postalcode,Borough,Neighborhood
0,M1A,Not assigned,
1,M2A,Not assigned,
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


<h5><font size=5> Data Cleaning and exploring</font></h5>

<h5>Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.</h5>

In [32]:
df=df[df['Borough']!='Not assigned']
df.head()

,Postalcode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


<h5>If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.</h5>

In [34]:
df.loc[df['Neighborhood'] =='Not assigned' , 'Neighborhood'] = df['Borough']
df.head()

,Postalcode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


<h5>rows with same postalcode have to be combined into one row seaparated by a comma.</h5>

In [37]:
result = df.groupby(['Postalcode','Borough'], sort=False).agg( ', '.join)
df_new=result.reset_index()
df_new.head()

,Postalcode,Borough,Neighborhood
0,M3A,North York,"Parkwoods, Parkwoods, Parkwoods"
1,M4A,North York,"Victoria Village, Victoria Village, Victoria V..."
2,M5A,Downtown Toronto,"Regent Park, Harbourfront, Regent Park, Harbou..."
3,M6A,North York,"Lawrence Manor, Lawrence Heights, Lawrence Man..."
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government, Q..."


<h5>Using the .shape method to print the number of rows of the new dataframe.</h5>

In [41]:
df_new.shape

(103, 3)